# Group Members
1. **Hassan Ahmed Shawon**
2. **Yeamin Sultana**
3. **Albin Israt**
4. **Mahmud Saleh Ove**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf



# Data preprocessing

# **Step 1**: Load the Dataset

---



In [ ]:
# Load CSV file
df = pd.read_csv("/content/drive/MyDrive/student_survey.csv")


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df.sample(10)

In [ ]:
df.isnull().sum()


# **Step 2:** Handle Missing Values


# **Step 3**: Encode Categorical Variables

In [ ]:

# Map categorical columns to numeric and **replace original columns**
df["Year_of_Study"] = df["Year_of_Study"].map({"1st":1, "2nd":2, "3rd":3, "4th":4})
df["Gender"] = df["Gender"].map({"Male":1, "Female":0})
df["Extracurricular_pariticipation"] = df["Extracurricular_pariticipation"].map({"Yes":1, "No":0})
df["PartTime_Job_Status"] = df["PartTime_Job_Status"].map({"Yes":1, "No":0})

# Remove rows where "Year of Study" is missing
df = df.dropna(subset=["Year_of_Study"])

# fill empty CGPA with CGPA Mean
df["CGPA"]= df["CGPA"].fillna(df["CGPA"].mean())
df["Extracurricular_Hours"]= df["Extracurricular_Hours"].fillna(value= 0)
df["PartTime_Job_Hours"]= df["PartTime_Job_Hours"].fillna(value=0)

# Summary statistics (include='all' shows both numbers + categorical columns)
print("\nSummary statistics:")
df.describe(include='all')


# Ensure No Missing Values

In [ ]:
df.isnull().sum()

# EDA
Univariate and Bivariate
1. Histograms / displot
2. Boxplots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric_cols = [
    "Age","CGPA","Year_of_Study","Study_Hours_per_Week","Sleep_Quality",
    "Sleep_Hours","Anxiety_Level","Stress_Level","PartTime_Job_Hours",
    "Extracurricular_Hours","Credits_Enrolled"
]

# Create a 5x5 grid
fig, axes = plt.subplots(5, 5, figsize=(20, 20))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.histplot(df[col].dropna(), bins=20, kde=True, ax=axes[i], color="#4ECDC4")
    axes[i].set_title(f"Distribution of {col}", fontsize=12)
    axes[i].set_xlabel(col, fontsize=10)
    axes[i].set_ylabel("Count", fontsize=10)

# Hide unused subplots
for j in range(len(numeric_cols), len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()


# Scatter Plots

In [ ]:

relationships = [
    ("Stress_Level", "Year_of_Study"),
    ("CGPA", "Gender"),
    ("Stress_Level", "PartTime_Job_Status"),
    ("Stress_Level", "Study_Hours_per_Week"),
    ("Sleep_Quality", "Anxiety_Level"),
    ("Sleep_Hours", "Anxiety_Level"),
    ("Anxiety_Level", "Year_of_Study")
]

fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (y, x) in enumerate(relationships):
    # Points colored by x (blue)
    axes[i].scatter(df[x], df[y], color='blue', alpha=0.6, s=50, label=f'{x} (x-axis)')

    # Overlay points colored by y (orange)
    axes[i].scatter(df[x], df[y], color='orange', alpha=0.4, s=50, label=f'{y} (y-axis)')

    axes[i].set_title(f"{y} vs {x}", fontsize=12)
    axes[i].set_xlabel(x, fontsize=10)
    axes[i].set_ylabel(y, fontsize=10)
    axes[i].legend(fontsize=8)

# Hide unused subplots
for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


# Detect Outliers

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

num_cols = df.select_dtypes(include=['number']).columns

fig, axes = plt.subplots(3, 3, figsize=(15, 12))  # adjust figsize as needed
axes = axes.flatten()  # flatten to 1D for easy indexing

for i, col in enumerate(num_cols[:9]):  # take first 9 numeric columns
    sns.boxplot(y=df[col], color="#4ECDC4", ax=axes[i])
    axes[i].set_title(f'Box Plot for {col}')

for j in range(len(num_cols), 9):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


Remove Outliers

Values below the lower bound are replaced with the lower bound.

Values above the upper bound are replaced with the upper bound.

In [ ]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Capping
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

In [ ]:
num_cols = df.select_dtypes(include=['number']).columns

for col in num_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(y=df[col], color='yellow')
    plt.title(f'Box Plot for {col}', fontsize=12, weight='bold')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# Feature Scaling by Standardization Process

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Heat Map for Correlation

In [ ]:

plt.figure(figsize=(18, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# These are our best 10 models based on Correlation HeatMap

In [ ]:

# SIMPLE MODELS
simple_models = [
    # 5 (strongest correlations)
    ('Year_of_Study', 'Age'),
    ('Extracurricular_Hours', 'Extracurricular_pariticipation'),
    ('PartTime_Job_Hours', 'PartTime_Job_Status'),
    ('Anxiety_Level', 'Stress_Level'),
    ('Sleep_Quality', 'Sleep_Hours'),

    # 5 (moderate to weak correlations)
    ('Year_of_Study', 'Stress_Level'),
    ('Anxiety_Level', 'Gender'),
    ('Sleep_Hours', 'Stress_Level'),
    ('CGPA', 'Age'),
    ('Sleep_Quality', 'Gender')
]


# MULTIPLE MODELS

multiple_models = [
    # 5 (based on higher correlations)
    ('Stress_Level', ['Anxiety_Level','Year_of_Study','Sleep_Hours']),
    ('Sleep_Quality', ['Sleep_Hours','Anxiety_Level','Stress_Level']),
    ('Anxiety_Level', ['Stress_Level','Sleep_Hours','Gender']),
    ('PartTime_Job_Hours', ['PartTime_Job_Status','Age','Sleep_Quality']),
    ('Extracurricular_Hours', ['Extracurricular_pariticipation','Year_of_Study','PartTime_Job_Status']),

    # 5 (medium correlations)
    ('Year_of_Study', ['Age','Stress_Level','PartTime_Job_Status']),
    ('Sleep_Hours', ['Stress_Level','Anxiety_Level','CGPA']),
    ('CGPA', ['Study_Hours_per_Week','Age','Sleep_Quality']),
    ('Credits_Enrolled', ['Year_of_Study','PartTime_Job_Status','Stress_Level']),
    ('Extracurricular_pariticipation', ['Extracurricular_Hours','Gender','Sleep_Hours'])
]


# Function to fit and print results
def fit_and_print(formula, df):
    model = smf.ols(formula=formula, data=df).fit()
    print("=== Model:", formula, "===")
    print(model.summary())
    print("\nR-squared: {:.3f}\n".format(model.rsquared))
    return model


# Run all models
simple_fitted = {}
print("\n### SIMPLE LINEAR MODELS ###\n")
for target, pred in simple_models:
    f = f"{target} ~ {pred}"
    simple_fitted[f] = fit_and_print(f, df)

multiple_fitted = {}
print("\n### MULTIPLE LINEAR MODELS ###\n")
for target, preds in multiple_models:
    rhs = " + ".join(preds)
    f = f"{target} ~ {rhs}"
    multiple_fitted[f] = fit_and_print(f, df)



In [ ]:

# ---- Function for simple regression scatterplot
def plot_simple_regression(ax, df, target, predictor, model):
    sns.scatterplot(x=df[predictor], y=df[target], color="blue", alpha=0.6, ax=ax, s=30)
    pred_vals = model.predict(df[[predictor]])
    sns.lineplot(x=df[predictor], y=pred_vals, color="red", ax=ax)
    ax.set_title(f"Regression: {target} vs {predictor}", fontsize=10)
    ax.set_xlabel(predictor, fontsize=8)
    ax.set_ylabel(target, fontsize=8)

# ---- Function for multiple regression scatterplot (Actual vs Predicted)
def plot_multiple_regression(ax, df, target, predictors, model):
    predicted = model.predict(df)
    sns.scatterplot(x=predicted, y=df[target], alpha=0.6, color="blue", ax=ax, s=30)
    min_val, max_val = df[target].min(), df[target].max()
    ax.plot([min_val, max_val], [min_val, max_val], color="red", linestyle="--")
    pred_str = " + ".join(predictors)  # use all predictor names
    ax.set_title(f"{target} ~ {pred_str}", fontsize=10)
    ax.set_xlabel("Predicted", fontsize=8)
    ax.set_ylabel("Actual", fontsize=8)


#  SIMPLE MODELS

fig, axes = plt.subplots(2, 5, figsize=(22, 8))
axes = axes.flatten()
for i, ((target, pred), model) in enumerate(zip(simple_models, simple_fitted.values())):
    plot_simple_regression(axes[i], df, target, pred, model)
plt.tight_layout()
plt.show()


# MULTIPLE MODELS
num_plots = len(multiple_models)
plots_per_fig = 4  # 2x2 grid
for start in range(0, num_plots, plots_per_fig):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    for i, idx in enumerate(range(start, min(start + plots_per_fig, num_plots))):
        (target, preds), model = multiple_models[idx], list(multiple_fitted.values())[idx]
        plot_multiple_regression(axes[i], df, target, preds, model)
    # Hide unused subplots
    for j in range(i+1, 4):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()


# Hypthesis Testing

In [ ]:

alpha = 0.05  # significance level

# -------------------------------
# 1. Independent t-test (CGPA by Gender)
# -------------------------------
male_cgpa = df[df["Gender"] == 0]["CGPA"]
female_cgpa = df[df["Gender"] == 1]["CGPA"]

t_stat, p_val = stats.ttest_ind(male_cgpa, female_cgpa, nan_policy="omit")

print("\n1.Independent t-test: CGPA by Gender")
print("Null hypothesis (H0): Mean CGPA of males = Mean CGPA of females")
print("Alternative hypothesis (H1): Mean CGPA of males ≠ Mean CGPA of females")
print()
print("t-statistic =", t_stat, ", p-value =", p_val)
print("Result:", "Reject H0 " if p_val < alpha else "Fail to reject H0 ")

# -------------------------------
# 2. One-way ANOVA (Stress_Level across Year_of_Study groups)
# -------------------------------
anova_groups = [group["Stress_Level"].dropna() for _, group in df.groupby("Year_of_Study")]

f_stat, p_val = stats.f_oneway(*anova_groups)

print("\n2.One-way ANOVA: Stress_Level ~ Year_of_Study")
print("Null hypothesis (H0): Mean Stress_Level is the same across all years of study")
print("Alternative hypothesis (H1): At least one year's mean Stress_Level is different")
print()
print("F-statistic =", f_stat, ", p-value =", p_val)
print("Result:", "Reject H0 " if p_val < alpha else "Fail to reject H0 ")

# -------------------------------
# 3. Correlation test (CGPA vs Anxiety_Level)
# -------------------------------
corr_coef, p_val = stats.pearsonr(df["CGPA"], df["Anxiety_Level"])

print("\n3.Correlation test: CGPA vs Anxiety_Level")
print("Null hypothesis (H0): No linear correlation between CGPA and Anxiety_Level (correlation = 0)")
print("Alternative hypothesis (H1): There is a linear correlation between CGPA and Anxiety_Level (correlation ≠ 0)")
print()
print("Correlation coefficient =", corr_coef, ", p-value =", p_val)
print("Result:", "Reject H0 " if p_val < alpha else "Fail to reject H0 ")

# -------------------------------
# 4. Proportion test (Chi-square: Gender vs Anxiety High/Low)
# -------------------------------
df["Anxiety_High"] = (df["Anxiety_Level"] > df["Anxiety_Level"].mean()).astype(int)
contingency_table = pd.crosstab(df["Gender"], df["Anxiety_High"])

chi2, p_val, dof , expected = stats.chi2_contingency(contingency_table)

print("\n4.Proportion test (Chi-square): Gender vs High Anxiety")
print("Null hypothesis (H0): Gender and High Anxiety are independent")
print("Alternative hypothesis (H1): Gender and High Anxiety are associated")
print()
print("Contingency table:\n", contingency_table)
print("Chi2 =", chi2, ", p-value =", p_val)
print("Result:", "Reject H0 " if p_val < alpha else "Fail to reject H0 ")


# APPlication

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
# ===============================
# Beginner-Friendly Dashboard in Colab (Streamlit)
# ===============================
%%writefile app.py
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import streamlit as st

# ===============================
# Load your cleaned dataframe
# ===============================
df = pd.read_csv("/content/drive/MyDrive/student_survey.csv")

if df["Year_of_Study"].dtype == "object":
    df["Year_of_Study"] = df["Year_of_Study"].map({"1st": 1, "2nd": 2, "3rd": 3, "4th": 4})

if df["Gender"].dtype == "object":
    df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})

if df["Extracurricular_pariticipation"].dtype == "object":
    df["Extracurricular_pariticipation"] = df["Extracurricular_pariticipation"].map({"Yes": 1, "No": 0})

if df["PartTime_Job_Status"].dtype == "object":
    df["PartTime_Job_Status"] = df["PartTime_Job_Status"].map({"Yes": 1, "No": 0})

#  Handle missing values
df = df.dropna(subset=["Year_of_Study"])
df["CGPA"] = df["CGPA"].fillna(df["CGPA"].mean())
df["Extracurricular_Hours"] = df["Extracurricular_Hours"].fillna(0)
df["PartTime_Job_Hours"] = df["PartTime_Job_Hours"].fillna(0)

# ===============================
# Sidebar Filters
# ===============================
st.sidebar.header("Filter Data")

year_filter = st.sidebar.selectbox("Year_of_Study", ["All"] + df["Year_of_Study"].unique().tolist())
gender_filter = st.sidebar.selectbox("Gender", ["All"] + df["Gender"].unique().tolist())
job_filter = st.sidebar.selectbox("PartTime_Job_Status", ["All"] + df["PartTime_Job_Status"].unique().tolist())

filtered_df = df.copy()
if year_filter != "All":
    filtered_df = filtered_df[filtered_df["Year_of_Study"] == year_filter]
if gender_filter != "All":
    filtered_df = filtered_df[filtered_df["Gender"] == gender_filter]
if job_filter != "All":
    filtered_df = filtered_df[filtered_df["PartTime_Job_Status"] == job_filter]

# ===============================
# Tab Bar
# ===============================
tabs = st.tabs([
    "Scatter + Regression",
    #"Boxplot",
    "Correlation Heatmap",
    "Simple Linear Models",
    "Multiple Linear Models"
])

# ===============================
# Tab 1: Scatter plot with regression line
# ===============================
with tabs[0]:
    st.subheader("Scatter Plot with Regression Line")
    numeric_cols = filtered_df.select_dtypes(include=['int64','float64']).columns.tolist()
    if len(numeric_cols) < 2:
        st.warning("Not enough numeric columns available for regression plot.")
    else:
        x_col = st.selectbox("Choose X (predictor):", numeric_cols, key="scatter_x")
        y_col = st.selectbox("Choose Y (target):", numeric_cols, key="scatter_y")
        fig, ax = plt.subplots()
        sns.regplot(x=x_col, y=y_col, data=filtered_df, ax=ax, ci=None)
        st.pyplot(fig)
    st.markdown("---")
    st.subheader(" / Interpretation")
    st.write("This dashboard shows simple Linear regression models scatterplot with regression line.")


# # ===============================
# # Tab 2: Boxplot
# # ===============================
# with tabs[1]:
#     st.subheader("Boxplot by Group")
#     group_col = st.selectbox("Group by:", df.columns, key="box_group")
#     value_col = st.selectbox("Value:", df.select_dtypes(include=['int64','float64']).columns.tolist(), key="box_value")
#     fig, ax = plt.subplots()
#     sns.boxplot(x=group_col, y=value_col, data=filtered_df, ax=ax)
#     st.pyplot(fig)

# ===============================
# Tab 3: Correlation Heatmap
# ===============================
with tabs[1]:
    st.subheader("Correlation Heatmap")
    corr = filtered_df.corr(numeric_only=True)
    fig, ax = plt.subplots(figsize=(12,8))
    sns.heatmap(corr, annot=True, cmap="coolwarm", ax=ax, fmt=".3f")
    st.pyplot(fig)
    st.markdown("---")
    st.subheader("Summary/ Interpretation")
    st.write("This dashboard shows the correlation using HeatMap."
              "This helps us to visualize the correlation among multiple features")

# ===============================
# Tab 4: Simple Linear Models
# ===============================
with tabs[2]:
    st.subheader("Simple Linear Models (10)")
    simple_models = [
        ('Year_of_Study', 'Age'),
        ('Extracurricular_Hours', 'Extracurricular_pariticipation'),
        ('PartTime_Job_Hours', 'PartTime_Job_Status'),
        ('Anxiety_Level', 'Stress_Level'),
        ('Sleep_Quality', 'Sleep_Hours'),
        ('Year_of_Study', 'Stress_Level'),
        ('Anxiety_Level', 'Gender'),
        ('Sleep_Hours', 'Stress_Level'),
        ('CGPA', 'Age'),
        ('Sleep_Quality', 'Gender')
    ]

    # Show titles
    model_options = [f"{y} ~ {x}" for y, x in simple_models]
    chosen_model = st.selectbox("Choose a Simple Model:", model_options)

    # Get correct (y, x) pair from selection
    y, x = simple_models[model_options.index(chosen_model)]

    if filtered_df[y].dtype == "object":
        st.error(f"Cannot run regression because dependent variable `{y}` is non-numeric.")
    else:
        # Handle categorical predictor
        if filtered_df[x].dtype == "object":
            formula = f"Q('{y}') ~ C(Q('{x}'))"
        else:
            formula = f"Q('{y}') ~ Q('{x}')"
        model = smf.ols(formula, data=filtered_df).fit()

        st.write("**Regression Equation:**")
        st.write(f"{y} = {model.params[0]:.4f} + {model.params[1]:.4f} * {x}")
        st.write("**Coefficients:**", model.params)
        st.write("**R²:**", model.rsquared)
    st.markdown("---")
    st.subheader("Summary")
    st.write("This dashboard shows the selected simple linear regression equation, r^2 , coefficient and intercept value.")


# ===============================
# Tab 5: Multiple Linear Models
# ===============================
with tabs[3]:
    st.subheader("Multiple Linear Models (10)")
    multiple_models = [
        ('Stress_Level', ['Anxiety_Level','Year_of_Study','Sleep_Hours']),
        ('Sleep_Quality', ['Sleep_Hours','Anxiety_Level','Stress_Level']),
        ('Anxiety_Level', ['Stress_Level','Sleep_Hours','Gender']),
        ('PartTime_Job_Hours', ['PartTime_Job_Status','Age','Sleep_Quality']),
        ('Extracurricular_Hours', ['Extracurricular_pariticipation','Year_of_Study','PartTime_Job_Status']),
        ('Year_of_Study', ['Age','Stress_Level','PartTime_Job_Status']),
        ('Sleep_Hours', ['Stress_Level','Anxiety_Level','CGPA']),
        ('CGPA', ['Study_Hours_per_Week','Age','Sleep_Quality']),
        ('Credits_Enrolled', ['Year_of_Study','PartTime_Job_Status','Stress_Level']),
        ('Extracurricular_pariticipation', ['Extracurricular_Hours','Gender','Sleep_Hours'])
    ]

    # Show titles
    model_options = [f"{y} ~ {' + '.join(Xs)}" for y, Xs in multiple_models]
    chosen_model = st.selectbox("Choose a Multiple Model:", model_options)

    # Get correct (y, Xs) pair from selection
    y, Xs = multiple_models[model_options.index(chosen_model)]

    if filtered_df[y].dtype == 'object':
        st.error(f"Cannot run regression because dependent variable `{y}` is non-numeric.")
    else:
        predictors = []
        for x in Xs:
            if filtered_df[x].dtype == 'object':
                predictors.append(f"C(Q('{x}'))")
            else:
                predictors.append(f"Q('{x}')")
        formula = f"Q('{y}') ~ {' + '.join(predictors)}"
        model = smf.ols(formula, data=filtered_df).fit()

        st.write("**Regression Equation:**")
        eqn = f"{y} = {model.params[0]:.4f} "
        for i, coef in enumerate(model.params[1:],1):
            eqn += f"+ {coef:.4f}*{Xs[i-1]} "
        st.write(eqn)
        st.write("**Coefficients:**", model.params)
        st.write("**R²:**", model.rsquared)

    st.markdown("---")
    st.subheader("Summary/ Interpretation")
    st.write("This dashboard shows the selected multiple linear regression equation, r^2 , coefficients and intercept values.")


In [ ]:
# Launch Streamlit with ngrok tunnel
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Replace with your token
NGROK_AUTH_TOKEN = "32eN4jnYOTeqprDbxQMwVwMvE9c_CD4exrJFA55RG12RMrfo"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("ngrok authentication token set.")

# Set up tunnel on port 8501
public_url = ngrok.connect(8501)
print("Streamlit app will be available at:", public_url)


# Run the app (make sure app_streamlit.py is in your current directory)
!streamlit run app.py --server.port 8501 > /dev/null 2>&1 &